## The Abstract Factory pattern

The Abstract Factory pattern is appropriate when we have multiple possible
implementations of a system that depend on some configuration or platform detail.
The calling code requests an object from the Abstract Factory, not knowing exactly
what class of object will be returned. The underlying implementation returned may
depend on a variety of factors, such as the current locale, operating system, or local
configuration.

![](uml/abstract_factory_pattern.png)

### An Abstract Factory example

![](uml/abstract_factory_example.png)

In [35]:
import abc
import collections
import random
from enum import Enum, auto
import itertools
from pprint import pprint
from typing import (
    Any,
    Counter,
    Iterator,
    Iterable,
    List,
    NamedTuple,
    TypeVar,
    cast,
)

class Suit(str, Enum):
    """
    >>> Suit.Spades
    <Suit.Spades: '♠'>
    """
    Clubs = "\N{Black Club Suit}"
    Diamonds = "\N{Black Diamond Suit}"
    Hearts = "\N{Black Heart Suit}"
    Spades = "\N{Black Spade Suit}"
    

class Card(NamedTuple):
    """
    >>> c = Card(5, Suit.Spades)
    >>> print(c)
    5♠
    >>> c
    Card(rank=5, suit=<Suit.Spades: '♠'>)
    """
    rank: int
    suit: Suit
    
    def __str__(self) -> str:
        return f"{self.rank}{self.suit.value}"
    
    
class CribbageCard(Card):
    @property
    def points(self) -> int:
        return self.rank
    
    
class CribbageAce(Card):
    @property
    def points(self) -> int:
        return 1
    
    
class CribbageFace(Card):
    @property
    def points(self) -> int:
        return 10

In [36]:
class Trick(int, Enum):
    pass


class CribbageTrick(Trick):
    """
    >>> list(CribbageTrick)[:2]
    [<CribbageTrick.Fifteen: 1>, <CribbageTrick.Pair: 2>]
    """
    Fifteen = auto()
    Pair = auto()
    Run_3 = auto()
    Run_4 = auto()
    Run_5 = auto()
    Right_Jack = auto()


In [72]:
c = TypeVar("C")

def powerset(iterable: Iterable[c]) -> Iterator[tuple[c, ...]]:
    """powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"""
    s = list(iterable)
    return itertools.chain.from_iterable(
        itertools.combinations(s, r) for r in range(len(s) + 1)
    )

cards = [
    CribbageCard(6, Suit.Clubs),
    CribbageCard(7, Suit.Diamonds),
    CribbageCard(8, Suit.Hearts),
]   

[tuple(map(str, t)) for t in powerset(cards)]

[(),
 ('6♣',),
 ('7♦',),
 ('8♥',),
 ('6♣', '7♦'),
 ('6♣', '8♥'),
 ('7♦', '8♥'),
 ('6♣', '7♦', '8♥')]

In [56]:
class Hand(List[Card]):
    """
    >>> Hand(Card(1, Suit.Spades), Card(5, Suit.Hearts))
    [Card(rank=1, suit=<Suit.Spades: '♠'>), Card(rank=5, suit=<Suit.Hearts: '♥'>)]
    """
    def __init__(self, *cards: Card) -> None:
        super().__init__(cards)
    
    @abc.abstractmethod
    def scoring(self) -> list[Trick]: ...
        
        
class CribbageHand(Hand):
    starter: Card
    
    def upcard(self, starter: Card) -> "Hand":
        self.starter = starter
        return self
    
    def scoring(self) -> list[Trick]:
        """15's. Pairs. Runs. Right Jack."""
        
        def trick_iter(cards: list[CribbageCard]) -> Iterator[Trick]:
            for subset in powerset(cards):
                if sum(card.points for card in subset) == 15:
                    yield CribbageTrick.Fifteen
            for card1, card2 in itertools.combinations(cards, 2):
                if card1.rank == card2.rank:
                    yield CribbageTrick.Pair
        
        def run_length(sorted_cards: list[CribbageCard]) -> int:
            card_iter = iter(sorted_cards)
            base = next(card_iter)
            for offset, card in enumerate(card_iter, start=1):
                if base.rank + offset != card.rank:
                    break
            return offset + 1
        
        hand_plus_starter = cast(List[CribbageCard], self + [self.starter])
        hand_plus_starter.sort()
        tricks = list(trick_iter(hand_plus_starter))
        
        if run_length(hand_plus_starter) == 5:
            tricks += [CribbageTrick.Run_5]
        elif (run_length(hand_plus_starter) == 4 
              or run_length(hand_plus_starter[1:]) == 4):
            tricks += [CribbageTrick.Run_4]
        elif (run_length(hand_plus_starter) == 3
              or run_length(hand_plus_starter[1:]) == 3
              or run_length(hand_plus_starter[2:]) == 3):
            tricks += [CribbageTrick.Run_3]
        
        right_jack = any(c.rank == 11 and c.suit == self.starter.suit
                         for c in self)
        if right_jack:
            tricks += [CribbageTrick.Right_Jack]
        return tricks
    
    
cards = [
    CribbageCard(6, Suit.Clubs),
    CribbageCard(7, Suit.Diamonds),
    CribbageCard(8, Suit.Hearts),
    CribbageCard(9, Suit.Spades),
]       
hand = CribbageHand(*cards)
print("Hand:")
pprint(list(map(str, hand)))
print()
print("Starter:")
starter = CribbageCard(5, Suit.Spades)
pprint(str(starter))
score = sorted(hand.upcard(starter).scoring())
list(map(lambda x: x.name, score))

Hand:
['6♣', '7♦', '8♥', '9♠']

Starter:
'5♠'


['Fifteen', 'Fifteen', 'Run_5']

In [52]:
class CardGameFactory(abc.ABC):
    @abc.abstractmethod
    def make_card(self, rank: int, suit: Suit) -> "Card": ...
        
    @abc.abstractmethod
    def make_hand(self, *cards: Card) -> "Hand": ...
        
        
class CribbageFactory(CardGameFactory):
    def make_card(self, rank: int, suit: Suit) -> "Card":
        if rank == 1:
            return CribbageAce(rank, suit)
        elif 2 <= rank < 11:
            return CribbageCard(rank, suit)
        else:
            return CribbageFace(rank, suit)
    
    def make_hand(self, *cards: Card) -> "Hand":
        return CribbageHand(*cards)
    
    
factory = CribbageFactory()
cards = [
    factory.make_card(6, Suit.Clubs),
    factory.make_card(7, Suit.Diamonds),
    factory.make_card(8, Suit.Hearts),
    factory.make_card(9, Suit.Spades),
]
starter = factory.make_card(5, Suit.Spades)
hand = factory.make_hand(*cards)
score = sorted(hand.upcard(starter).scoring())
[t.name for t in score]

['Fifteen', 'Fifteen', 'Run_5']

In [94]:
class PokerCard(Card):
    def __str__(self) -> str:
        if self.rank == 14:
            return f"A{self.suit.value}"
        return f"{self.rank}{self.suit.value}"

class PokerTrick(Trick):
    Pair = auto()
    TwoPair = auto()
    Three = auto()
    Straight = auto()
    Flush = auto()
    FullHouse = auto()
    Four = auto()
    StraightFlush = auto()
    
    
class PokerHand(Hand):
    def scoring(self) -> list[Trick]:
        """Return a single 'Trick'"""
        # Disctinc Ranks -> Counter({5: 3, 6: 2})
        ranks: Counter[int] = collections.Counter(c.rank for c in self)
        # Dictinct Suits -> True if all cards have the same Suit
        flush = len(set(c.suit for c in self)) == 1
        if len(ranks) == 1:
            # five of a kind!
            raise Exception(f"Broken Hand {self}")
        elif len(ranks) == 2:
            # 4-1 or 3-2 ['1♣', '5♦', '5♥', '5♠', '5♠']
            card, count = ranks.most_common(1)[0]
            if count == 4:
                return [PokerTrick.Four]
            elif count == 3:
                return [PokerTrick.FullHouse]
            else:
                raise Exception(f"Broken Hand {self}")
        elif len(ranks) == 3:
            # 3-1-1, or 2-2-1 ['1♣', '2♦', '5♥', '5♠', '5♠']
            card, count = ranks.most_common(1)[0]
            if count == 3:
                return [PokerTrick.Three]
            elif count == 2:
                return [PokerTrick.TwoPair]
            else:
                raise Exception(f"Broken Hand {self}")
        elif len(ranks) == 4:
            # 2-1-1-1 ['1♣', '2♦', '3♥', '5♠', '5♠']
            return [PokerTrick.Pair]
        elif len(ranks) == 5:
            # straight ? ['1♣', '2♦', '3♥', '4♠', '5♠']
            base = min(ranks)
            straight = all(base + offset == rank
                           for offset, rank in enumerate(ranks))
            
            if straight and flush:  # ['1♦', '2♦', '3♦', '4♦', '5♦']
                return [PokerTrick.StraightFlush]
            elif straight:  # ['1♣', '2♦', '3♥', '4♠', '5♠']
                return [PokerTrick.Straight]
            elif flush:  # ['1♦', '2♦', '3♦', '4♦', '7♦']
                return [PokerTrick.Flush]
            else:
                return []
        else:
            return []
            
cards = [
    PokerCard(1, Suit.Clubs),
    PokerCard(2, Suit.Diamonds),
    PokerCard(3, Suit.Hearts),
    PokerCard(4, Suit.Spades),
    PokerCard(5, Suit.Spades),
]

hand = PokerHand(*cards)
pprint(list(map(str, hand)))
print()
hand.scoring()

['1♣', '2♦', '3♥', '4♠', '5♠']



[<PokerTrick.Straight: 4>]

In [95]:
class PokerFactory(CardGameFactory):
    def make_card(self, rank: int, suit: Suit) -> "Card":
        if rank == 1:
            # Aces above kings
            rank = 14
        return PokerCard(rank, suit)
    def make_hand(self, *cards: Card) -> "Hand":
        return PokerHand(*cards)
    
factory = PokerFactory()
cards = [
    factory.make_card(5, Suit.Clubs),
    factory.make_card(5, Suit.Diamonds),
    factory.make_card(5, Suit.Hearts),
    factory.make_card(6, Suit.Spades),
    factory.make_card(6, Suit.Spades),
]
hand = factory.make_hand(*cards)
hand.scoring()

[<PokerTrick.FullHouse: 6>]

In [114]:
class Game:
    def __init__(self, factory: CardGameFactory) -> None:
        self.factory = factory
        
    def prepare(self) -> None:
        self.deck = [
            self.factory.make_card(r, s) 
            for r in range(1, 14)
            for s in iter(Suit)
        ]
        random.shuffle(self.deck)
    
    def deal(self) -> "Hand":
        hand = self.factory.make_hand(*self.deck[:5])
        return hand
    
    def score(self, hand: Hand) -> None:
        print(hand.scoring())
        
class Poker(Game):
    pass


random.seed(278)
poker_game = Poker(PokerFactory())
poker_game.prepare()
hand = poker_game.deal()
print(list(map(str, hand)))
poker_game.score(hand)

['2♣', '13♥', '13♣', '13♠', '2♥']
[<PokerTrick.FullHouse: 6>]


In [116]:
class Cribbage(Game):
    def score(self, hand: Hand) -> None:
        up_card = self.deck[5]
        hand = cast(CribbageHand, hand).upcard(up_card)
        print(hand.scoring())
        

cribbage_game = Cribbage(CribbageFactory())
cribbage_game.prepare()
hand = cribbage_game.deal()
print(list(map(str, hand)))
cribbage_game.score(hand)

['6♦', '5♠', '10♠', '11♥', '7♣']
[<CribbageTrick.Fifteen: 1>, <CribbageTrick.Fifteen: 1>, <CribbageTrick.Pair: 2>, <CribbageTrick.Run_3: 3>]
